## Version20 : Focal-CTC-Loss(0.75,0.5) + lstm-dropout(0.1,0.1)  + LeakyReLU(0.05) + augement + Adam(0.001,1e-3) + Freeze(1~4) + data_merge(1560) + batch(64)

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import editdistance
import pickle
import time

from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint, TensorBoard

from crnn_model_focal_ctc_loss import CRNN
from crnn_data_focal_ctc_loss import InputGenerator
from crnn_utils import decode
from utils.training import Logger, ModelSnapshot
from cracker_util import cracker_dict

Using TensorFlow backend.
/home/ina/anaconda3/envs/ina/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:768: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 2. Skipping tag 41493
  " Skipping tag %s" % (size, len(data), tag))
/home/ina/anaconda3/envs/ina/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:768: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 41988
  " Skipping tag %s" % (size, len(data), tag))


In [3]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

plt.rcParams['axes.labelsize'] = 18
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 16

#matplotlib 한글 폰트 설정

path = '/usr/share/fonts/truetype/nanum/NanumGothicEco.ttf'
font_name = fm.FontProperties(fname=path, size=16).get_name()
print(font_name)
plt.rc('font', family=font_name)
mpl.rcParams['axes.unicode_minus'] = False
fm._rebuild()

NanumGothic Eco


In [4]:
from data_cracker import GTUtility

file_name = 'gt_util_merge_data.pkl'

with open(file_name, 'rb') as f:
    gt_util_cracker = pickle.load(f)

In [5]:
gt_util_train, gt_util_val = GTUtility.randomSplit(gt_util_cracker)

In [6]:
print(len(gt_util_cracker.image_names))
print(len(gt_util_train.image_names))
print(len(gt_util_val.image_names))

1520
1216
304


In [7]:
from cracker_util import cracker_dict
from crnn_utils import alphabet87
print(len(alphabet87))
print(len(cracker_dict))

87
836


In [8]:
input_width = 256
input_height = 32
batch_size = 32
input_shape = (input_width, input_height, 1)
max_string_len = 62

In [9]:
experiment = 'crnn_ksignboard_test'

## ---predict---

model, model_pred = CRNN(input_shape, len(cracker_dict), gru=False, alpha=0.75,gamma=0.5)
max_string_len = model_pred.output_shape[1]

## ---training---

In [13]:
alpha=0.75
gamma=0.5

model, model_pred = CRNN(input_shape, len(cracker_dict), gru=False, alpha=alpha,gamma=gamma)
model_trained,_ = CRNN(input_shape, len(alphabet87), gru=False, alpha=alpha,gamma=gamma)

model_trained.load_weights('weights.h5')

max_string_len = model_pred.output_shape[1]

Instructions for updating:
Colocations handled automatically by placer.


/home/ina/anaconda3/envs/ina/lib/python3.7/site-packages/keras/activations.py:211: UserWarning: Do not pass a layer instance (such as LeakyReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.


In [14]:
for idx in range(16):
    extracted_weights = model_trained.layers[idx].get_weights()
    model.layers[idx].set_weights(extracted_weights)

In [15]:
freeze = ['conv1_1',
          'conv2_1',
          'conv3_1', 'conv3_2', 
          'conv4_1',
          #'conv5_1',
          #'conv6_1',
          #'lstm1',
          #'lstm2'
         ]


In [16]:
gen_train = InputGenerator(gt_util_train, batch_size, cracker_dict, input_shape[:2], 
                           grayscale=True, max_string_len=max_string_len)
gen_val = InputGenerator(gt_util_val, batch_size, cracker_dict, input_shape[:2], 
                         grayscale=True, max_string_len=max_string_len)

In [17]:
checkdir = './checkpoints/' + time.strftime('%Y%m%d%H%M') + '_' + experiment
if not os.path.exists(checkdir):
    os.makedirs(checkdir)

with open(checkdir+'/source.py','wb') as f:
    source = ''.join(['# In[%i]\n%s\n\n' % (i, In[i]) for i in range(len(In))])
    f.write(source.encode())
    
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=32, 
                                          write_graph=True, write_grads=False, write_images=False, 
                                          embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, 
                                          embeddings_data=None, update_freq='epoch')

# tensorboard --logdir=./logs

# livelossplot

from livelossplot.keras import PlotLossesCallback

#EarlyStopping
from keras.callbacks import EarlyStopping

In [ ]:
loss = hist.history['loss']
val_loss = hist.history['val_loss']

epochs = range(len(loss))
plt.figure(figsize=(15,10))
plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b',label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
model_pred.load_weights('./checkpoints/201904301143_crnn_ksignboard_v20/weights.057.h5')

In [ ]:
g = gen_val.generate()

In [ ]:
d = next(g)

In [ ]:
d[0]['source_str'].shape

In [ ]:
res = model_pred.predict(d[0]['image_input'])

mean_ed = 0
mean_ed_norm = 0

plot_name = 'crnn_ksignboard'

#for i in range(len(res)):
all_total_length = 0
all_score = 0
for i in range(32):
    
    # best path, real ocr applications use beam search with dictionary and language model
    chars = [cracker_dict[c] for c in np.argmax(res[i], axis=1)]
    gt_str = d[0]['source_str'][i]
    res_str = decode(chars)
    
    ed = editdistance.eval(gt_str, res_str)
    #ed = levenshtein(gt_str, res_str)
    ed_norm = ed / len(gt_str)
    mean_ed += ed
    mean_ed_norm += ed_norm
    
    g = list(gt_str)
    r = list(res_str)
    score = 0
    if len(g) == len(r):
        total_len = len(g)   
        for idx in range(total_len):
            if g[idx] == r[idx]:
                score += 1
        
        acc = (score / total_len ) * 100
        all_score += score
        all_total_length += total_len
    else:
        total_len = len(g)
        for idx in range(len(r)):
            if r[idx] in g:
                score += 1
        acc = (score / total_len ) * 100
        all_score += score
        all_total_length += total_len
        
    # display image
    img = d[0]['image_input'][i][:,:,0].T
    plt.figure(figsize=[10,1.03])
    plt.imshow(img, cmap='gray', interpolation=None)
    ax = plt.gca()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    plt.text(0, 45, '%s' % (''.join(chars)) )
    plt.text(0, 60, 'GT: %-24s RT: %-24s %0.2f   %0.2f' % (gt_str, res_str, ed_norm, acc))
    
            
    
    #file_name = 'plots/%s_recogniton_%03d.pgf' % (plot_name, i)
    file_name = 'plots/%s_recogniton_%03d.png' % (plot_name, i)
    #plt.savefig(file_name, bbox_inches='tight', dpi=300)
    #print(file_name)
    
    plt.show()
    
    #print('%-20s %-20s %s %0.2f' % (gt_str, res_str, ''.join(chars), ed_norm))

mean_ed /= len(res)
mean_ed_norm /= len(res)
all_acc = (all_score / all_total_length ) * 100

print('\nmean editdistance: %0.3f\nmean normalized editdistance: %0.3f\naccuracy: %0.3f' % (mean_ed, mean_ed_norm, all_acc))